In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *


fig_DIR = "../figs/team_ratings/"
box_DIR = "./boxscores/"

In [ ]:
season = "2023"
df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
df['Team_Split'] = df['MATCHUP'].str.replace("@","")
df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
df[["FG3M","PLUS_MINUS","FG3_PCT"]] = df[["FG3M","PLUS_MINUS","FG3_PCT"]].astype(str)

In [ ]:
df.columns

# Based on 3FGM

In [ ]:
df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
df1[["FG3M_For", "FG3M_Against"]] = df1["FG3M"].str.split(",", expand=True)
df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3M","PLUS_MINUS"]].agg(",".join).reset_index()
df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
df2[["FG3M_Against","FG3M_For"]] = df2["FG3M"].str.split(",", expand=True)
df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)

In [ ]:
df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
df3 = df3[["Team1","Team2","FG3M_For", "FG3M_Against","PM1","PM2"]]
df3.iloc[:,2:] = df3.iloc[:,2:].astype(int)

In [ ]:
df3["More_3PT"] = df3["FG3M_For"] > df3["FG3M_Against"]
df3["Less_3PT"] = df3["FG3M_For"] <= df3["FG3M_Against"]
df3["Win"] = df3["PM1"] > 0
df3["Loss"] = df3["PM1"] < 0
df3["Win_More_3PT"] = df3["More_3PT"] & df3["Win"]
df3["Win_Less_3PT"] = df3["Less_3PT"] & df3["Win"]
df3["Loss_More_3PT"] = df3["More_3PT"] & df3["Loss"]
df3["Loss_Less_3PT"] = df3["Less_3PT"] & df3["Loss"]

In [ ]:
df4 = df3.groupby("Team1")[["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"]]\
.agg({"Win_More_3PT":["sum"],"Win_Less_3PT":["sum"],"Loss_More_3PT":["sum"],"Loss_Less_3PT":["sum"]})
df4.columns = ["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"]
df4 = df4.sort_values(by="Win_More_3PT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
df4.loc['Total']= df4.sum()
df4.loc['Total','Team'] = "Total"

In [ ]:
d = (
    gt.GT(data=df4.iloc[:15])
    # .cols_label(For_30= "30 PTS For", Against_30="30 PTS Against", Diff_30="Difference")
    .cols_align(align="center", columns=["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"])
    .tab_spanner(
        label = "More 3PT",
        columns = ["Win_More_3PT", "Loss_More_3PT"]
    )
    .tab_spanner(
        label = "Less 3PT",
        columns = ["Win_Less_3PT", "Loss_Less_3PT"]
    )
    .tab_header(
        title="Team Win Loss When Making More 3 Pointers than Opponent",
    )
    .cols_label(Win_More_3PT = "Wins",Win_Less_3PT = "Wins", Loss_More_3PT = "Losses",Loss_Less_3PT = "Losses")
    .tab_source_note(
        source_note = "@SravanNBA | source: nba.com/stats"
    )
)
d

In [ ]:
hti.screenshot(html_str=d._render_as_html(), save_as="test1.png", size=(600, 750))

In [ ]:
d = (
    gt.GT(data=df4.iloc[15:])
    # .cols_label(For_30= "30 PTS For", Against_30="30 PTS Against", Diff_30="Difference")
    .cols_align(align="center", columns=["Win_More_3PT","Win_Less_3PT","Loss_More_3PT","Loss_Less_3PT"])
    .tab_spanner(
        label = "More 3PT",
        columns = ["Win_More_3PT", "Loss_More_3PT"]
    )
    .tab_spanner(
        label = "Less 3PT",
        columns = ["Win_Less_3PT", "Loss_Less_3PT"]
    )
    .tab_header(
        title="Team Win Loss When Making More 3 Pointers than Opponent",
    )
    .cols_label(Win_More_3PT = "Wins",Win_Less_3PT = "Wins", Loss_More_3PT = "Losses",Loss_Less_3PT = "Losses")
    .tab_source_note(
        source_note = "@SravanNBA | source: nba.com/stats"
    )
)
d

In [ ]:
hti.screenshot(html_str=d._render_as_html(), save_as="test2.png", size=(600, 750))

# Based on 3PT%

In [ ]:
df1 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
df1[["Team1", "Team2"]] = df1["TEAM_ABBREVIATION"].str.split(",", expand=True)
df1[["FG3_PCT_For", "FG3_PCT_Against"]] = df1["FG3_PCT"].str.split(",", expand=True)
df1[["PM1", "PM2"]] = df1["PLUS_MINUS"].str.split(",", expand=True)
df2 = df.groupby("GAME_ID")[["TEAM_ABBREVIATION","FG3_PCT","PLUS_MINUS"]].agg(",".join).reset_index()
df2[["Team2", "Team1"]] = df2["TEAM_ABBREVIATION"].str.split(",", expand=True)
df2[["FG3_PCT_Against","FG3_PCT_For"]] = df2["FG3_PCT"].str.split(",", expand=True)
df2[["PM2", "PM1"]] = df2["PLUS_MINUS"].str.split(",", expand=True)

In [ ]:
df3 = pd.concat([df1,df2]).sort_values(by="GAME_ID").reset_index(drop=True)
df3 = df3[["Team1","Team2","FG3_PCT_For", "FG3_PCT_Against","PM1","PM2"]]
df3.iloc[:,2:] = df3.iloc[:,2:].astype(float)

In [ ]:
df3["More_3PT_PCT"] = df3["FG3_PCT_For"] > (df3["FG3_PCT_Against"] +0.1)
df3["Win"] = df3["PM1"] > 0
df3["Loss"] = df3["PM1"] < 0
df3["Win_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Win"]
df3["Loss_More_3PT_PCT"] = df3["More_3PT_PCT"] & df3["Loss"]

In [ ]:
df4 = df3.groupby("Team1")[["Win_More_3PT_PCT","Loss_More_3PT_PCT"]]\
.agg({"Win_More_3PT_PCT":["sum"],"Loss_More_3PT_PCT":["sum"]})
df4.columns = ["Win_More_3PT_PCT","Loss_More_3PT_PCT"]
df4 = df4.sort_values(by="Win_More_3PT_PCT",ascending=False).reset_index().rename(columns={"Team1":"Team"})
df4.loc['Total']= df4.sum()
df4.loc['Total','Team'] = "Total"

In [ ]:
d = (
    gt.GT(data=df4)
    # .cols_label(For_30= "30 PTS For", Against_30="30 PTS Against", Diff_30="Difference")
    .cols_align(align="center", columns=["Win_More_3PT_PCT","Loss_More_3PT_PCT"])
    .tab_spanner(
        label = "3PT % > Opponent by 10%",
        columns = ["Win_More_3PT_PCT", "Loss_More_3PT_PCT"]
    )
    .tab_header(
        title="Team Win Loss When 3PT% greater than Opponent by 10%",
    )
    .cols_label(Win_More_3PT_PCT = "Wins",Loss_More_3PT_PCT = "Losses")
    .tab_source_note(
        source_note = "@SravanNBA | source: nba.com/stats"
    )
)
d

In [ ]:
hti.screenshot(html_str=d._render_as_html(), save_as="test3.png", size=(400, 1400))